In [56]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PoissonRegressor
from patsy import dmatrices

# funciones desarrolladas
from functions.agrupar_dfs_censo import *
from functions.cargar_data import *
from functions.impresion import *

In [57]:
dd_deptos = pd.read_csv('tablas/dd_deptos.csv')

In [67]:
y, X = dmatrices('personas_mig ~ nom_depto_orig + log_pbi_destino + log_dist -1', dd_deptos, return_type = 'dataframe')

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [83]:
X_train.shape, X_test.shape

((256, 21), (86, 21))

In [84]:
# alpha=0.0 determina el no uso de rgularización
model = PoissonRegressor(alpha=0, fit_intercept = False, max_iter=10000)
mdl = model.fit(X_train, y_train.values.ravel())
np.around(model.coef_, 4)

array([ 1.0073,  1.0011,  0.379 ,  0.1267, -0.2219, -1.0867, -0.5827,
       -0.4268,  0.061 ,  2.4526,  0.5047, -0.1048,  0.7142, -0.0301,
        0.7867, -0.6386,  0.208 ,  0.6341,  0.0712,  0.901 , -0.8508])

In [79]:
print(model.score(X, y.values.ravel()))

0.9430610101435695


In [86]:
y_pred = model.predict(X_test)

In [87]:
y_pred

array([6.62088529e+01, 3.30903568e+01, 1.28647679e+02, 2.08434949e+01,
       1.19484087e+02, 3.48328305e+01, 1.16749855e+02, 4.09100253e+01,
       7.95205087e+01, 6.87306306e+01, 4.42989389e+01, 2.02442030e+02,
       8.50815940e+01, 3.48287937e+04, 1.24476474e+02, 2.19271918e+02,
       6.74419708e+01, 8.98951551e+01, 1.10048353e+02, 8.26381044e+01,
       3.03860820e+02, 3.75098489e+01, 5.92152283e+02, 1.08337482e+02,
       1.69111747e+02, 7.40731294e+01, 3.77319866e+02, 1.24968913e+02,
       1.88898725e+02, 5.44862991e+01, 5.51143517e+01, 1.20986203e+02,
       4.65835214e+01, 2.66232224e+02, 5.39732480e+01, 8.56306389e+01,
       4.50666562e+03, 9.28142434e+02, 5.27195711e+01, 3.76209036e+01,
       3.79545389e+02, 9.39575501e+01, 1.06409508e+02, 6.33986021e+02,
       7.52819861e+02, 4.05375615e+01, 1.58364593e+02, 7.57375010e+01,
       6.61966539e+02, 1.51410886e+02, 4.84505449e+01, 8.20728685e+01,
       5.61326537e+01, 9.72598668e+01, 7.44940771e+01, 1.32749640e+02,
      